In [1]:
from pyspark.sql import SparkSession

In [59]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
spark = (SparkSession
        .builder
        .appName("SparkSQLExampleApp")
        .config("spark.sql.catalogImplementation", "hive")
        .getOrCreate())

In [3]:
csv_file = "./flights/departuredelays.csv"

In [4]:
# Read and create a temporary view
# Infer schema (note that for larger files you
# may want to specify the schema)
df = (spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .load(csv_file))

In [5]:
df.show(10)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+
only showing top 10 rows



In [6]:
df.createOrReplaceTempView("us_delay_flights_tbl")

In [7]:
# In Python
# schema = "'date' STRING, 'delay' INT, 'distance' INT,'origin' STRING, 'destination' STRING"

In [8]:
spark.sql("""DESC us_delay_flights_tbl""").show()

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|       date|      int|   NULL|
|      delay|      int|   NULL|
|   distance|      int|   NULL|
|     origin|   string|   NULL|
|destination|   string|   NULL|
+-----------+---------+-------+



In [9]:
spark.sql("""select * from us_delay_flights_tbl limit 10""").show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+



In [10]:
spark.sql("""SELECT distance, origin, destination
FROM us_delay_flights_tbl WHERE distance > 1000
ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [11]:
spark.sql("""SELECT date, delay, origin, destination
FROM us_delay_flights_tbl
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
ORDER by delay DESC""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [12]:
spark.sql("""SELECT date, concat(substr(date, 1, 1), '-', 
substr(date, 2, 2), ' ', substr(date, 4, 2), ':', substr(date, 6, 2), ' ') DATE,
delay, origin, destination
FROM us_delay_flights_tbl
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
ORDER by delay DESC""").show(10)

+-------+-----------+-----+------+-----------+
|   date|       DATE|delay|origin|destination|
+-------+-----------+-----+------+-----------+
|2190925|2-19 09:25 | 1638|   SFO|        ORD|
|1031755|1-03 17:55 |  396|   SFO|        ORD|
|1022330|1-02 23:30 |  326|   SFO|        ORD|
|1051205|1-05 12:05 |  320|   SFO|        ORD|
|1190925|1-19 09:25 |  297|   SFO|        ORD|
|2171115|2-17 11:15 |  296|   SFO|        ORD|
|1071040|1-07 10:40 |  279|   SFO|        ORD|
|1051550|1-05 15:50 |  274|   SFO|        ORD|
|3120730|3-12 07:30 |  266|   SFO|        ORD|
|1261104|1-26 11:04 |  258|   SFO|        ORD|
+-------+-----------+-----+------+-----------+
only showing top 10 rows



In [13]:
# converting the date column into a readable format and find
# months when these delays were most common.

spark.sql("""SELECT substr(date, 1, 1) month, count(substr(date, 1, 1)) as count_month from
(SELECT concat(substr(date, 1, 1), '-', 
substr(date, 2, 2), ' ', substr(date, 4, 2), ':', substr(date, 6, 2), ' ') date,
delay, origin, destination
FROM us_delay_flights_tbl
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
ORDER by delay DESC)
GROUP BY month order by count_month desc
""").show(10)

+-----+-----------+
|month|count_month|
+-----+-----------+
|    1|         22|
|    2|         17|
|    3|         16|
+-----+-----------+



In [14]:
# converting the date column into a readable format and find
# the days when these delays were most common.

spark.sql("""SELECT substr(date, 3, 2) day, count(substr(date, 1, 1)) count_day from
(SELECT concat(substr(date, 1, 1), '-', 
substr(date, 2, 2), ' ', substr(date, 4, 2), ':', substr(date, 6, 2), ' ') date,
delay, origin, destination
FROM us_delay_flights_tbl
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
ORDER by delay DESC)
GROUP BY day order by count_day desc
""").show(10)

+---+---------+
|day|count_day|
+---+---------+
| 02|        5|
| 03|        4|
| 31|        4|
| 30|        3|
| 26|        3|
| 05|        3|
| 27|        3|
| 17|        3|
| 09|        3|
| 12|        3|
+---+---------+
only showing top 10 rows



In [15]:
spark.sql("""SELECT delay, origin, destination,
CASE
WHEN delay > 360 THEN 'Very Long Delays'
WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
WHEN delay = 0 THEN 'No Delays'
ELSE 'Early'
END AS Flight_Delays
FROM us_delay_flights_tbl
ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [16]:
from pyspark.sql.functions import col, desc
(df.select("distance", "origin", "destination")
.where(col("distance") > 1000)
.orderBy(desc("distance"))).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [17]:
# Or
(df.select("distance", "origin", "destination")
.where("distance > 1000")
.orderBy("distance", ascending=False).show(10))

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [18]:
# Or
spark.sql("""SELECT distance, origin, destination
FROM us_delay_flights_tbl WHERE distance > 1000
ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



### Creating SQL Databases and Tables

In [19]:
spark.sql("""CREATE DATABASE IF NOT EXISTS learn_spark_db""")

DataFrame[]

In [20]:
spark.sql("USE learn_spark_db")

DataFrame[]

In [21]:
spark.sql("SHOW TABLES").show()

+--------------+--------------------+-----------+
|     namespace|           tableName|isTemporary|
+--------------+--------------------+-----------+
|learn_spark_db|managed_us_delay_...|      false|
|learn_spark_db|managed_us_delay_...|      false|
|              |us_delay_flights_tbl|       true|
+--------------+--------------------+-----------+



In [22]:
# spark.sql("""CREATE TABLE IF NOT EXISTS managed_us_delay_flights_tbl (date STRING, delay INT,
# distance INT, origin STRING, destination STRING)""")

In [24]:
spark.sql("""DROP TABLE IF EXISTS managed_us_delay_flights_tbl""")

DataFrame[]

In [25]:
# In Python
# Path to our US flight delays CSV file
csv_file = "./flights/departuredelays.csv"
# Schema as defined in the preceding example
schema="date STRING, delay INT, distance INT, origin STRING, destination STRING"
flights_df = spark.read.csv(csv_file, schema=schema)
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

In [30]:
spark.sql("""SELECT * FROM managed_us_delay_flights_tbl""").show(10)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01201755|    0|     449|   ORF|        ATL|
|01201610|   52|     449|   ORF|        ATL|
|01201441|    0|     449|   ORF|        ATL|
|01211755|  -15|     449|   ORF|        ATL|
|01210941|   -5|     449|   ORF|        ATL|
|01210700|    5|     449|   ORF|        ATL|
|01211243|    2|     449|   ORF|        ATL|
|01210540|   -5|     449|   ORF|        ATL|
|01211610|   27|     449|   ORF|        ATL|
|01211441|   -6|     449|   ORF|        ATL|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [27]:
spark.sql("""DESC managed_us_delay_flights_tbl""").show()

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|       date|   string|   NULL|
|      delay|      int|   NULL|
|   distance|      int|   NULL|
|     origin|   string|   NULL|
|destination|   string|   NULL|
+-----------+---------+-------+



In [28]:
spark.sql("""DESCRIBE managed_us_delay_flights_tbl""").show()

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|       date|   string|   NULL|
|      delay|      int|   NULL|
|   distance|      int|   NULL|
|     origin|   string|   NULL|
|destination|   string|   NULL|
+-----------+---------+-------+



In [29]:
spark.sql("""CREATE TABLE IF NOT EXISTS managed_us_delay_flights_tbl2 (date STRING, delay INT,
distance INT, origin STRING, destination STRING)""")

DataFrame[]

### Creating an unmanaged table

In [33]:
spark.sql("""CREATE TABLE IF NOT EXISTS us_delay_flights_tbl_unmanaged (date STRING, delay INT,
distance INT, origin STRING, destination STRING)
USING csv OPTIONS (PATH
'./flights/departuredelays.csv')""")

Py4JJavaError: An error occurred while calling o26.sql.
: java.lang.UnsatisfiedLinkError: 'org.apache.hadoop.io.nativeio.NativeIO$POSIX$Stat org.apache.hadoop.io.nativeio.NativeIO$POSIX.stat(java.lang.String)'
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.stat(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.getStat(NativeIO.java:608)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.loadPermissionInfoByNativeIO(RawLocalFileSystem.java:934)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.loadPermissionInfo(RawLocalFileSystem.java:848)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.getGroup(RawLocalFileSystem.java:832)
	at org.apache.hadoop.hive.io.HdfsUtils.setFullFileStatus(HdfsUtils.java:102)
	at org.apache.hadoop.hive.io.HdfsUtils.setFullFileStatus(HdfsUtils.java:94)
	at org.apache.hadoop.hive.io.HdfsUtils.setFullFileStatus(HdfsUtils.java:77)
	at org.apache.hadoop.hive.common.FileUti

In [34]:
(flights_df
.write
.option("path", "/tmp/data/us_flights_delay")
.saveAsTable("us_delay_flights_tbl"))

In [35]:
(flights_df
.write
.option("path", "/tmp/data/us_flights_delay")
.saveAsTable("us_delay_flights_tbl_unmanaged"))

AnalysisException: CREATE-TABLE-AS-SELECT cannot create table with location to a non-empty directory file:///tmp/data/us_flights_delay. To allow overwriting the existing non-empty directory, set 'spark.sql.legacy.allowNonEmptyLocationInCTAS' to true.

In [40]:
spark.sql("""CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view AS
SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE
origin = 'SFO';""").show()

++
||
++
++



In [41]:
spark.sql("""CREATE OR REPLACE TEMP VIEW us_origin_airport_JFK_tmp_view AS
SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE
origin = 'JFK';""").show()

++
||
++
++



In [78]:
# In Python
df_sfo = spark.sql("""SELECT date, delay, origin, destination FROM
us_delay_flights_tbl WHERE origin = 'SFO';""")

In [79]:
df_sfo.show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1011250|   55|   SFO|        JFK|
|1012230|    0|   SFO|        JFK|
|1010705|   -7|   SFO|        JFK|
|1010620|   -3|   SFO|        MIA|
|1010915|   -3|   SFO|        LAX|
|1011005|   -8|   SFO|        DFW|
|1011800|    0|   SFO|        ORD|
|1011740|   -7|   SFO|        LAX|
|1012015|   -7|   SFO|        LAX|
|1012110|   -1|   SFO|        MIA|
+-------+-----+------+-----------+
only showing top 10 rows



In [46]:
df_jfk = spark.sql("""SELECT date, delay, origin, destination FROM
us_delay_flights_tbl WHERE origin = 'JFK';""")

In [47]:
df_jfk.show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1010900|   14|   JFK|        LAX|
|1011200|   -3|   JFK|        LAX|
|1011900|    2|   JFK|        LAX|
|1011700|   11|   JFK|        LAS|
|1010800|   -1|   JFK|        SFO|
|1011540|   -4|   JFK|        DFW|
|1011705|    5|   JFK|        SAN|
|1011530|   -3|   JFK|        SFO|
|1011630|   -3|   JFK|        SJU|
|1011345|    2|   JFK|        LAX|
+-------+-----+------+-----------+
only showing top 10 rows



In [80]:
# Create a temporary and global temporary view
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")

In [87]:
# Create a temporary and global temporary view
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

In [57]:
spark.sql("""SHOW TABLES;""").show()

+--------------+--------------------+-----------+
|     namespace|           tableName|isTemporary|
+--------------+--------------------+-----------+
|learn_spark_db|managed_us_delay_...|      false|
|learn_spark_db|managed_us_delay_...|      false|
|learn_spark_db|us_delay_flights_tbl|      false|
|              |us_delay_flights_tbl|       true|
|              |us_origin_airport...|       true|
+--------------+--------------------+-----------+



In [90]:
spark.sql("""SHOW VIEWS;""").show()

+---------+--------------------+-----------+
|namespace|            viewName|isTemporary|
+---------+--------------------+-----------+
|         |us_delay_flights_tbl|       true|
|         |us_origin_airport...|       true|
|         |us_origin_airport...|       true|
+---------+--------------------+-----------+



In [82]:
spark.sql("""SELECT * FROM global_temp.us_origin_airport_SFO_global_tmp_view;""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1011250|   55|   SFO|        JFK|
|1012230|    0|   SFO|        JFK|
|1010705|   -7|   SFO|        JFK|
|1010620|   -3|   SFO|        MIA|
|1010915|   -3|   SFO|        LAX|
|1011005|   -8|   SFO|        DFW|
|1011800|    0|   SFO|        ORD|
|1011740|   -7|   SFO|        LAX|
|1012015|   -7|   SFO|        LAX|
|1012110|   -1|   SFO|        MIA|
+-------+-----+------+-----------+
only showing top 10 rows



In [91]:
spark.sql("""SELECT * FROM us_origin_airport_JFK_tmp_view ;""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1010900|   14|   JFK|        LAX|
|1011200|   -3|   JFK|        LAX|
|1011900|    2|   JFK|        LAX|
|1011700|   11|   JFK|        LAS|
|1010800|   -1|   JFK|        SFO|
|1011540|   -4|   JFK|        DFW|
|1011705|    5|   JFK|        SAN|
|1011530|   -3|   JFK|        SFO|
|1011630|   -3|   JFK|        SJU|
|1011345|    2|   JFK|        LAX|
+-------+-----+------+-----------+
only showing top 10 rows



In [96]:
# Python
spark.read.table("us_origin_airport_JFK_tmp_view").show(5)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1010900|   14|   JFK|        LAX|
|1011200|   -3|   JFK|        LAX|
|1011900|    2|   JFK|        LAX|
|1011700|   11|   JFK|        LAS|
|1010800|   -1|   JFK|        SFO|
+-------+-----+------+-----------+
only showing top 5 rows



In [97]:
# Or
spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view").show(5)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|1010900|   14|   JFK|        LAX|
|1011200|   -3|   JFK|        LAX|
|1011900|    2|   JFK|        LAX|
|1011700|   11|   JFK|        LAS|
|1010800|   -1|   JFK|        SFO|
+-------+-----+------+-----------+
only showing top 5 rows



In [103]:
spark.sql("DROP VIEW IF EXISTS global_temp.us_origin_airport_SFO_global_tmp_view;").show()

++
||
++
++



In [100]:
spark.sql("DROP VIEW IF EXISTS us_origin_airport_JFK_tmp_view").show()

++
||
++
++



In [104]:
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")

False

In [102]:
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

False

In [107]:
spark.catalog.listDatabases()

[Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='file:/D:/OneDrive%2520-%2520xpdnp/Coding/Data%2520Science/Books/Learning%2520Spark%25202nd%2520edition/mine/chapter4/spark-warehouse'),
 Database(name='learn_spark_db', catalog='spark_catalog', description='', locationUri='file:/D:/OneDrive%2520-%2520xpdnp/Coding/Data%2520Science/Books/Learning%2520Spark%25202nd%2520edition/mine/chapter4/spark-warehouse/learn_spark_db.db')]

In [108]:
spark.catalog.listTables()

[Table(name='managed_us_delay_flights_tbl', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='managed_us_delay_flights_tbl2', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='us_delay_flights_tbl', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='us_delay_flights_tbl', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='us_origin_airport_JFK_tmp_view2', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [109]:
spark.catalog.listColumns("us_delay_flights_tbl")

[Column(name='date', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

## Caching SQL Tables

In [ ]:
# -- In SQL
# CACHE [LAZY] TABLE <table-name>
# UNCACHE TABLE <table-name>

## Reading Tables into DataFrames

In [110]:
# In Python
us_flights_df = spark.sql("SELECT * FROM us_delay_flights_tbl")
us_flights_df2 = spark.table("us_delay_flights_tbl")

In [111]:
us_flights_df.show(5)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+
only showing top 5 rows



In [112]:
us_flights_df2.show(5)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+
only showing top 5 rows



# Data Sources for DataFrames and SQL Tables

## DataFrameReader

In [113]:
# DataFrameReader.format(args).option("key", "value").schema(args).load()

In [114]:
# SparkSession.read
# // or
# SparkSession.readStream

## DataFrameWriter

In [ ]:
# DataFrameWriter.format(args)
# .option(args)
# .bucketBy(args)
# .partitionBy(args)
# .save(path)

# DataFrameWriter.format(args).option(args).sortBy(args).saveAsTable(table)

In [ ]:
# DataFrame.write
# // or
# DataFrame.writeStream

## Parquet

### Reading Parquet files into a DataFrame

In [118]:
!echo %cd%

D:\OneDrive - xpdnp\Coding\Data Science\Books\Learning Spark 2nd edition\mine\chapter4


In [123]:
# In Python
file = """flights/summary-data/parquet/2010-summary.parquet"""

In [124]:
df = spark.read.format("parquet").load(file)

In [125]:
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



### Reading Parquet files into a Spark SQL table

In [128]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
USING parquet OPTIONS (path "flights/summary-data/parquet/2010-summary.parquet/" );""")

DataFrame[]

In [131]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
USING parquet OPTIONS (path "flights/summary-data/parquet/2010-summary.parquet/" );""").show(5)

++
||
++
++



In [132]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows

